In [3]:
dadagp_path = '/Users/pedro/Desktop/DadaGP-v1.1/'
all_files_path = '/Users/pedro/Desktop/DadaGP-v1.1/_DadaGP_all_filenames.json'

## Get all .gp files

In [12]:
import json

# Assuming 'all_files_path' contains the path to your JSON file
with open(all_files_path, 'r') as f:
    file_paths = json.load(f)  # Load JSON data into a Python list

# Remove the ".tokens.txt" part from each path
all_gps_paths = [path.replace(".tokens.txt", "") for path in file_paths]

# Print the number of files
print(len(all_gps_paths))

26181


## Get all .gp files with => 4 tracks

In [25]:
import guitarpro as gp
import os
from tqdm import tqdm

gp_files_4plus = []

# Iterate over the list of .gp file paths
for gp_file_path in tqdm(all_gps_paths):
    # Get full path
    file_path = os.path.join(dadagp_path, gp_file_path)
    # Parse them with pyguitarpro
    try:
        # Parse the Guitar Pro file
        gp_file = gp.parse(file_path)

        # Check the number of tracks
        if len(gp_file.tracks) >= 4:
            gp_files_4plus.append(file_path)
            # print('File {} has {} tracks.'.format(file_path, len(gp_file.tracks)))

    except Exception as e:
        # Handle any parsing errors gracefully
        print('Could not parse {}: {}'.format(file_path, e))
    

100%|██████████| 26181/26181 [30:38<00:00, 14.24it/s]  


In [33]:
print('There are {} files with 4 or more tracks, which is {:.2f}% of the dataset.'.format(len(gp_files_4plus), 100*len(gp_files_4plus)/len(all_gps_paths)))

There are 13413 files with 4 or more tracks, which is 51.23% of the dataset.


In [34]:
# Save gp_files_4plus into a .json
with open("gp_paths_4plustracks.json", "w") as final:
	json.dump(gp_files_4plus, final)

## Remove extra tracks from .gp files

In [38]:
# If a track only has 2 distorted guitars, bass, drums -> save the track
# If a track has more than those instruments, check if it has 2 DGs, B, D -> save those in new file

with open('gp_paths_4plustracks.json', 'r') as f:
    gp_files_4plus = json.load(f)  # Load JSON data into a Python list

13413


In [218]:
gp_file = gp.parse(gp_files_4plus[14])
_ = gp_file
filename = gp_files_4plus[14]

# Initialize lists to store required instruments
distorted_guitars = []
bass = None
drums = None

for track in gp_file.tracks:
        print(track.name, track.channel.instrument)
        instrument_type = is_required_instrument(track)
        if instrument_type == 'distorted_guitar' and len(distorted_guitars) < 2:
            print('dg')
            distorted_guitars.append(track)
        elif instrument_type == 'bass' and bass is None:
            print('bass')
            bass = track
        elif instrument_type == 'drums' and drums is None:
            print('drums')
            drums = track

print(distorted_guitars)

if len(distorted_guitars) == 2 and bass and drums:
    required_tracks = distorted_guitars + [bass, drums]
    print(required_tracks)

    _.tracks = []
    for track in required_tracks:
        # print(track)
        _.tracks.append(track)
    gp.write(_, 'a.gp5')
    print("Created new file with 4 tracks: {}".format(output_directory + filename))

Track 1 25
Track 2 32
bass
Track 3 27
Percussion 0
drums
Percussion 0
[]


In [219]:
import os
import shutil
from tqdm import tqdm

# Directory to save filtered files
output_directory = "/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/"

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# MIDI Channel numbers for the required instruments
DISTORTED_GUITAR_CHANNELS = [29, 30]  # Distorted/Overdriven guitars
BASS_CHANNEL = [32, 33, 34, 35, 36, 37, 38, 39]  # Finger/Picked Bass
# No need to define drums channel as we'll use isPercussionTrack for this

# Function to check if a track is a distorted guitar, bass, or drums
def is_required_instrument(track):
    instrument = track.channel.instrument
    if track.isPercussionTrack:  # Check if it's a percussion (drum) track
        return 'drums'
    elif instrument in DISTORTED_GUITAR_CHANNELS:
        return 'distorted_guitar'
    elif instrument in BASS_CHANNEL:
        return 'bass'
    
    return None

for file in tqdm(gp_files_4plus):
    # print(file)
    try:
        # Parse the Guitar Pro file
        # print(gp_files_4plus[666])
        gp_file = gp.parse(file)
        _ = gp_file
        # print(gp_file)
        filename = file.split('/')[-1]
        print(filename)

        # Initialize lists to store required instruments
        distorted_guitars = []
        bass = None
        drums = None

        # Filter tracks based on MIDI channel and percussion check
        for track in gp_file.tracks:
            # print(track)
            instrument_type = is_required_instrument(track)
            if instrument_type == 'distorted_guitar' and len(distorted_guitars) < 2:
                distorted_guitars.append(track)
            elif instrument_type == 'bass' and bass is None:
                bass = track
            elif instrument_type == 'drums' and drums is None:
                drums = track

        # Check if we have the required tracks (2 distorted guitars, 1 bass, 1 drums)
        if len(distorted_guitars) == 2 and bass and drums:
            required_tracks = distorted_guitars + [bass, drums]
            # print(required_tracks)

            if len(gp_file.tracks) == 4:
                # If the file already has exactly 4 tracks, just copy it
                output_file = os.path.join(output_directory, filename)
                shutil.copy(file, output_file)
                print("Copied {} to {}.".format(file_path, output_file))
            else:
                # _ was a copy of the current .gp file with all tracks
                # we need to clear its tracks
                _.tracks = []
                for track in required_tracks:
                    # print(track)
                    _.tracks.append(track)
                gp.write(_, output_directory + filename)
                print("Created new file with 4 tracks: {}".format(output_directory + filename))

    except Exception as e:
        print("Could not process {}: {}".format(gp_file, e))



  0%|          | 1/13413 [00:00<26:14,  8.52it/s]

12 Stones - Crash.gp4
Created new file with 4 tracks: /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/12 Stones - Crash.gp4
704 exercises for guitar, 1-96 - Beast.gp3
Copied /Users/pedro/Desktop/DadaGP-v1.1/Z/Zyklon/Zyklon - Psyklon Aeon.gp4 to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/704 exercises for guitar, 1-96 - Beast.gp3.
1974 AD, Pahilo Junema - Jungi Nishan.gp4


  0%|          | 7/13413 [00:00<22:27,  9.95it/s]

1Kempfi1 - 4 Seasons.gp4
1Kempfi1 - Short & Stupid.gp4
Copied /Users/pedro/Desktop/DadaGP-v1.1/Z/Zyklon/Zyklon - Psyklon Aeon.gp4 to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/1Kempfi1 - Short & Stupid.gp4.
1Kempfi1 - Sunrise.gp4
1st Man For Her - A Peach Fantasy.gp4
2 Unlimited - No One (2).gp3


  0%|          | 11/13413 [00:01<37:50,  5.90it/s]

...

  1%|          | 151/13413 [00:20<53:18,  4.15it/s]

ACDC - For Those About To Rock.gp3
Created new file with 4 tracks: /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/ACDC - For Those About To Rock.gp3
ACDC - Hard As A Rock (2).gp3


  1%|          | 153/13413 [00:20<42:50,  5.16it/s]

ACDC - Hard As A Rock (2).gp4
Created new file with 4 tracks: /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/ACDC - Hard As A Rock (2).gp4
ACDC - Hard As A Rock.gp3
Copied /Users/pedro/Desktop/DadaGP-v1.1/Z/Zyklon/Zyklon - Psyklon Aeon.gp4 to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4/ACDC - Hard As A Rock.gp3.


  1%|          | 155/13413 [00:20<32:50,  6.73it/s]